##신입기수 교육: RNN Assignment
: text generation using RNN

: 데이터 이해/전처리를 읽고 난 후 모델 및 Text Generation 함수를 설계해보기

: 참고) text generation using RNN.ipynb파일, https://wikidocs.net/45101


**1) 데이터 이해/전처리**

In [ ]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [ ]:
# ArticlesApril2018.csv 파일 로드
df = pd.read_csv('ArticlesApril2018.csv')

In [ ]:
print('Number of columns: ', len(df.columns))
print(df.columns)

Number of columns:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [ ]:
df['headline'].isnull().values.any()

False

In [ ]:
# 모든 기사의 제목 하나의 리스트로 저장
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [ ]:
# headline 중 unknown 처리

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [ ]:
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafaring Past, and Muddied Its Future',
 'A Quote Disproved',
 'Hot Stuff Turns Cold',
 'At the Top,

In [ ]:
import re

# 데이터 전처리: 구두점 제거, lowercase()
def data_preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    return text

cnt=0
for i in headline:
    headline[cnt] = data_preprocessing(i)
    cnt+=1
headline[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift its still looking',
 'romney failed to win at utah convention but few believe hes doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now hes meeting with pope francis',
 'in berlin artists find a home',
 'the right stuff',
 'jimmy carter knows what north korea wants',
 'the truth is out there',
 'new jersey ruling could reignite battle over churchstate separation',
 'procrastinating',
 'word  quiz dilatory',
 'my lifethreatening bout with e coli food poisoning',
 'choosing brexit a town yearned for its seafaring past and muddied its future',
 'a quote disproved',
 'hot stuff turns cold',
 'at the top the pay gap may be gone',
 'year

In [ ]:
# 토큰화 및 단어 집합 크기 확인
t= Tokenizer()
t.fit_on_texts(headline)
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 3494


In [ ]:
# 정수 인코딩 & 하나의 문장을 2 이상 길이의 훈련 데이터로 만들기"
sequences = []
for text in headline:
    for line in text.split('\n'):    # \n 기준으로 문장 토큰화
        encoded = t.texts_to_sequences([line])[0]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
            sequences.append(sequence)

cnt = 0
for i in sequences:
    if cnt < 11:
        cnt+=1
        print(i)

[99, 269]
[99, 269, 371]
[99, 269, 371, 1115]
[99, 269, 371, 1115, 582]
[99, 269, 371, 1115, 582, 52]
[99, 269, 371, 1115, 582, 52, 7]
[99, 269, 371, 1115, 582, 52, 7, 2]
[99, 269, 371, 1115, 582, 52, 7, 2, 372]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116]
[100, 3]


In [ ]:
print(sequences)

[[99, 269], [99, 269, 371], [99, 269, 371, 1115], [99, 269, 371, 1115, 582], [99, 269, 371, 1115, 582, 52], [99, 269, 371, 1115, 582, 52, 7], [99, 269, 371, 1115, 582, 52, 7, 2], [99, 269, 371, 1115, 582, 52, 7, 2, 372], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], [100, 3], [100, 3, 1117], [100, 3, 1117, 2], [100, 3, 1117, 2, 14], [100, 3, 1117, 2, 14, 583], [100, 3, 1117, 2, 14, 583, 24], [100, 3, 1117, 2, 14, 583, 24, 1118], [100, 3, 1117, 2, 14, 583, 24, 1118, 373], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5, 1119], [1, 14], [1, 14, 1120], [1, 14, 1120, 1121], [15, 2], [15, 2, 584], [15, 2, 584, 4], [15, 2, 584, 4, 215], [15, 2, 584, 4, 215, 375], [15, 2, 584, 4, 215, 375, 1122], [15, 2, 584, 4, 215, 375, 1122, 2], [15, 2, 584, 4, 215, 375, 1122, 2, 376], [15, 2, 584, 4, 215, 375, 1122, 2, 376, 585], [8, 167], [8, 167, 2], [8,

In [ ]:
# 빈도수 상위 단어 확인
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 576번 단어 : {}'.format(index_to_word[576]))
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 576번 단어 : victims
빈도수 상위 1번 단어 : the


In [ ]:
# 가장 긴 샘플의 길이에 전체 샘플 길이 맞추기
max_len = max(len(l) for l in sequences) 
print('샘플 최대 길이 : {}'.format(max_len))

# 전체 샘플 길이를 max_len으로 패딩 작업
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print()
print('패딩 결과 확인')
print(sequences)

샘플 최대 길이 : 24

패딩 결과 확인
[[   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 [   0    0    0 ...  269  371 1115]
 ...
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]
 [   0    0    0 ...  115    2 1025]]


In [ ]:
# 훈련 데이터에서 레이블 분리 (각 문장의 마지막 단어만 분리해서 y로)
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print('X 확인')
print(X)
print()
print('y 확인')
print(y)

# 레이블에 대한 one-hot encoding
y = to_categorical(y, num_classes = vocab_size)

print()
print('one-hot encoding : y 확인')
print(y)

X 확인
[[   0    0    0 ...    0    0   99]
 [   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 ...
 [   0    0    0 ...  170    8 3493]
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]]

y 확인
[ 269  371 1115 ...  115    2 1025]

one-hot encoding : y 확인
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**2) 모델 설계하기**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model = Sequential()

model.add(# FILL IN) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model.add(# FILL IN)   # 모델 및 은닉 상태 크기 설정 
model.add(# FILL IN) #Dense Layer 및 활성화 함수 설정
          
model.compile(         ,         , metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정 

model.fit(X,y,epochs = 50, verbose=2) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 

Epoch 1/50
244/244 - 7s - loss: 7.6708 - accuracy: 0.0293 - 7s/epoch - 27ms/step
Epoch 2/50
244/244 - 3s - loss: 7.0791 - accuracy: 0.0338 - 3s/epoch - 13ms/step
Epoch 3/50
244/244 - 3s - loss: 6.9007 - accuracy: 0.0399 - 3s/epoch - 11ms/step
Epoch 4/50
244/244 - 3s - loss: 6.7371 - accuracy: 0.0490 - 3s/epoch - 11ms/step
Epoch 5/50
244/244 - 3s - loss: 6.5851 - accuracy: 0.0531 - 3s/epoch - 11ms/step
Epoch 6/50
244/244 - 3s - loss: 6.4412 - accuracy: 0.0545 - 3s/epoch - 11ms/step
Epoch 7/50
244/244 - 3s - loss: 6.2967 - accuracy: 0.0598 - 3s/epoch - 14ms/step
Epoch 8/50
244/244 - 3s - loss: 6.1510 - accuracy: 0.0675 - 3s/epoch - 11ms/step
Epoch 9/50
244/244 - 3s - loss: 6.0023 - accuracy: 0.0734 - 3s/epoch - 11ms/step
Epoch 10/50
244/244 - 3s - loss: 5.8571 - accuracy: 0.0824 - 3s/epoch - 11ms/step
Epoch 11/50
244/244 - 3s - loss: 5.7140 - accuracy: 0.0877 - 3s/epoch - 11ms/step
Epoch 12/50
244/244 - 3s - loss: 5.5753 - accuracy: 0.0930 - 3s/epoch - 11ms/step
Epoch 13/50
244/244 - 3s 

In [ ]:
# 문장 생성 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # 데이터에 대한 패딩
        predict_x=model.predict(encoded, verbose=2) 
        classes_x=np.argmax(predict_x,axis=1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == classes_x: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
# Text Generation 결과 확인 
# 임의의 단어 'good'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'good', 10))

1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 26ms/epoch - 26ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
good going to be besties a kidney agent and other national
